In [ ]:
from __future__ import print_function
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

import numpy as np
np.set_printoptions(threshold = np.nan, linewidth = 115)
import pickle
import keras

from keras import models, optimizers, layers, regularizers, metrics, losses
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU, ReLU, ThresholdedReLU
from keras.layers.core import Dense, Dropout, SpatialDropout2D
from keras.layers.convolutional import Conv2D, SeparableConv2D
from keras.layers.pooling import MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import model_from_json, Sequential

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config = config)


IMG_SIZE = 256

# Save Comparison model
def save_model(model_name, hist_str, model_str):

    pickle.dump(model_name.history, open('Training Histories/'+ hist_str + '.p', 'wb'))
    
    print("Saved " + hist_str + " to Training Histories folder")
    
    # serialize model to JSON
    model_name = model.to_json()
    with open("CNN Models/" + model_str + ".json", "w") as json_file:
        json_file.write(model_name)

    # serialize weights to HDF5
    model.save_weights("CNN Models/" + model_str + ".h5")
    print("Saved " + model_str + " and weights to CNN Models folder")
    

# Define data() and create_model() methods
def data():
    # Load with pickle instead of processing images again
    training_img_1 = pickle.load(open('training_img_1.p', 'rb'))
    training_img_2 = pickle.load(open('training_img_2.p', 'rb'))
    training_img = np.append(training_img_1, training_img_2, axis = 0)
    training_labels = pickle.load(open('training_labels.p', 'rb'))
    
    test_img = pickle.load(open('pickle_test_img.p', 'rb'))
    test_labels = pickle.load(open('test_labels.p', 'rb'))
    
    return training_img, training_labels, test_img, test_labels

# Metric Analysis
def true_accuracy(pred, y_test):
    
    pred = pred.astype(dtype = 'int32')
    
    ft = pred == y_test
    
    acc = []
    
    counter = 0
    while counter < len(ft):
        if sum(ft[counter]) < 15:
            acc.append(0)
            counter += 1
        else:
            acc.append(1)
            counter += 1
            
    # Accuracy       
    Acc = (sum(acc)/len(acc))
    
    print('\t Complete Label Accuracy: ', (Acc * 100), '%')
    
    print('Sum of Fully Correct Predictions: ', sum(acc))
    print('\t\t    Total Labels: ', len(acc))
    
    if Acc == 0:
        message = 'Feels Devastating (ﾉಥ益ಥ）ﾉ ┻━┻'
        
    elif Acc > 0 and Acc < 50:
        message = 'Feels Awful (੭ ˃̣̣̥ ㅂ˂̣̣̥)੭ु'
        
    elif Acc >= 50 and Acc < 60:
        message = 'Feels Bad (⌯˃̶᷄ ﹏ ˂̶᷄⌯)'
        
    elif Acc >= 60 and Acc < 70:
        message = 'Feels Meh... ┬─┬ノ(ಠ_ಠノ)'
    
    elif Acc >= 70 and Acc < 80:
        message = 'Feels Ok ʕ ·㉨·ʔ'
    
    elif Acc >= 80 and Acc < 90:
        message = 'Feels Better (^._.^)ﾉ'
        
    elif Acc >= 90 and Acc < 95:
        message = 'Feels Hopeful ( •́ ⍨ •̀)'
        
    elif Acc >= 95 and Acc < 98:
        message = 'Feels Good ヽ|･ω･|ゞ'
        
    elif Acc >= 98:
        message = 'Feels Great! ᕙ( * •̀ ᗜ •́ * )ᕗ'
        
    print('\n', message)
    
    return Acc

def create_model(training_img, training_labels, test_img, test_labels):

    # Vanilla ConvNets
    model = Sequential()

    # Block 1
    model.add(Conv2D({{choice([16, 32, 64])}}, 
                     {{choice([(3, 3), (5, 5), (7, 7)])}}, 
                     kernel_regularizer = regularizers.l2({{uniform(0, 0.01)}}), 
                     input_shape = (IMG_SIZE, IMG_SIZE, 1)))
    model.add({{choice([ReLU, LeakyReLU, PReLU, ELU, ThresholdedReLU])}})

    model.add(Conv2D({{choice([16, 32, 64])}}, 
                     {{choice([(3, 3), (5, 5)])}}, 
                     kernel_regularizer = regularizers.l2({{uniform(0, 0.01)}})))
    model.add({{choice([ReLU, LeakyReLU, PReLU])}})

    if {{choice(['MaxPool', 'Global'])}} == 'MaxPool':

        model.add(MaxPooling2D(pool_size = {{choice([2, 4])}}, 
                               strides = {{choice([1, 2])}}))

    model.add({{choice([Dropout({{choice([(0.25, 0.375, 0.5)])}}), 
                        SpatialDropout2D({{choice([(0.25, 0.375, 0.5)])}})])}})
              
    # Block 2
    model.add(Conv2D({{choice([16, 32, 64])}}, 
                     {{choice([(3, 3), (5, 5)])}}, 
                     kernel_regularizer = regularizers.l2({{uniform(0, 0.01)}})))
    model.add({{choice([ReLU, LeakyReLU, PReLU])}})

    model.add(Conv2D({{choice([16, 32, 64])}}, 
                     {{choice([(3, 3), (5, 5)])}}, 
                     kernel_regularizer = regularizers.l2({{uniform(0, 0.01)}})))
    model.add({{choice([ReLU, LeakyReLU, PReLU])}})

    if {{choice(['two', 'three'])}} == 'three':

        model.add(Conv2D({{choice([16, 32, 64])}}, 
                     {{choice([(3, 3), (5, 5)])}}, 
                     kernel_regularizer = regularizers.l2({{uniform(0, 0.01)}})))
        model.add({{choice([ReLU, LeakyReLU, PReLU])}})

    if {{choice(['MaxPool', 'Global'])}} == 'MaxPool':

        model.add(MaxPooling2D(pool_size = {{choice([2, 4])}}, 
                               strides = {{choice([1, 2])}}))

    model.add({{choice([Dropout({{choice([(0.25, 0.375, 0.5)])}}), 
                        SpatialDropout2D({{choice([(0.25, 0.375, 0.5)])}})])}})


    # Block 3
    model.add(Conv2D({{choice([64, 128, 256])}}, 
                     {{choice([(3, 3), (5, 5)])}}, 
                     kernel_regularizer = regularizers.l2({{uniform(0, 0.01)}})))
    model.add({{choice([ReLU, LeakyReLU, PReLU])}})

    model.add(Conv2D({{choice([64, 128, 256])}}, 
                     {{choice([(3, 3), (5, 5)])}}, 
                     kernel_regularizer = regularizers.l2({{uniform(0, 0.01)}})))
    model.add({{choice([ReLU, LeakyReLU, PReLU])}})

    if {{choice(['two', 'three'])}} == 'three':

        model.add(Conv2D({{choice([64, 128, 256])}}, 
                     {{choice([(3, 3), (5, 5)])}}, 
                     kernel_regularizer = regularizers.l2({{uniform(0, 0.01)}})))
        model.add({{choice([ReLU, LeakyReLU, PReLU])}})

    if {{choice(['MaxPool', 'Global'])}} == 'MaxPool':

        model.add(MaxPooling2D(pool_size = {{choice([2, 4])}}, 
                               strides = {{choice([1, 2])}}))

    model.add({{choice([Dropout({{choice([(0.25, 0.375, 0.5)])}}), 
                        SpatialDropout2D({{choice([(0.25, 0.375, 0.5)])}})])}})


    # Output Block

    if {{choice(['MaxPool', 'Global'])}} == 'Global':

        model.add(GlobalAveragePooling2D())

    model.add(Dense({{choice([16, 32, 64])}}))
    model.add({{choice([ReLU, LeakyReLU, PReLU])}})

    if {{choice(['one', 'two'])}} == 'two':

        model.add(Dense({{choice([16, 32, 64])}}))
        model.add({{choice([ReLU, LeakyReLU, PReLU])}})

    model.add(Dense(15))
    model.add(Activation({{choice(['sigmoid', 'softmax'])}}))

    model.compile(loss = 'binary_crossentropy', 
                  optimizer = optimizers.RMSprop(lr = {{uniform(0.0001, 0.001)}}), 
                  metrics = ['categorical_accuracy'])

    model.fit(training_img, training_labels,
              batch_size = 64,
              epochs = 10,
              verbose = 2,
              validation_split = 0.1)

    Predictions = model.predict(test_img)
    Predictions = Predictions.astype(dtype = 'int32')

    Accuracy = true_accuracy(Predictions, test_labels)

    return {'loss': -Accuracy, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model=optim.minimize(model = create_model, data = data, algo = tpe.suggest, max_evals = 20, trials = Trials(), notebook_name = 'Neural_Network_Architectures')

    X_train, Y_train, X_test, Y_test = data()
    print('Scores of best performing model:\n')
    best_predict = best_model.predict(X_test)
    true_accuracy(best_predict, Y_test)
    print('Best performing model chosen hyper-parameters:\n')
    print(best_run)

# Hyperas Model Engineering


def create_vanilla(training_img, training_labels, test_img, test_labels):
    
    # Model-Wide Choices
    Activation_Choice = {{choice([ReLU, LeakyReLU, PReLU])}}
    Regularizer_Choice = regularizers.l2({{uniform(0, 0.01)}})
    Filters_Choice = {{choice([4, 8, 16])}}    
    Kernel_Size_Choice = {{choice([(3, 3), (5, 5)])}}
    Pool_Choice = {{choice(['MaxPool', 'Global'])}}
    Max_Pool_Size_Choice = {{choice([2, 4])}}
    Strides_Choice = {{choice([1, 2])}}
    Dropout_Choice = {{choice([Dropout, SpatialDropout2D])}}
    Block_1_Output = {{choice([16, 32, 64])}}
    
    if Block_1_Output == 16:
        
        Block_2_Output = {{choice([16, 32, 64])}}
        Block_3_Output = {{choice([32, 64, 128])}}
        
    elif Block_1_Output == 32:
        
        Block_2_Output = {{choice([32, 64, 128])}}
        Block_3_Output = {{choice([64, 128, 256])}}
        
    else:
        
        Block_2_Output = {{choice([64, 128])}}
        Block_3_Output = {{choice([128, 256])}}
        
    B1_Conv_Layer_Choice = {{choice([Conv2D(Block_1_Output, Kernel_Size_Choice), 
                                  SeparableConv2D(Filters_Choice, Kernel_Size_Choice)])}}
    
    B2_Conv_Layer_Choice = {{choice([Conv2D(Block_2_Output, Kernel_Size_Choice), 
                                  SeparableConv2D(Filters_Choice, Kernel_Size_Choice)])}}
    
    B3_Conv_Layer_Choice = {{choice([Conv2D(Block_3_Output, Kernel_Size_Choice), 
                                  SeparableConv2D(Filters_Choice, Kernel_Size_Choice)])}}
    
    Dense_Neurons_Choice = {{choice([16, 32, 64])}}
    
    if Dense_Neurons_Choice > 32 and Block_3_Output > 128:
        
        Batch_Size_Choice = 64
        
    else:
        Batch_Size_Choice = 128
    
    Learning_Rate_Choice = {{uniform(0.0001, 0.001)}}
    
    
    # Vanilla ConvNets
    model = Sequential()
    
    # Block 1
    model.add(Conv2D(Block_1_Output, 
                     {{choice([(3, 3), (5, 5), (7, 7)])}}, 
                     kernel_regularizer = Regularizer_Choice, 
                     input_shape = (IMG_SIZE, IMG_SIZE, 1)))
    model.add({{choice([ReLU, LeakyReLU, PReLU, ELU, ThresholdedReLU])}})
    
    model.add(B1_Conv_Layer_Choice)
    model.add(Activation_Choice)
    
    if Pool_Choice == 'MaxPool':
        
        model.add(MaxPooling2D(pool_size = Max_Pool_Size_Choice, 
                               strides = Strides_Choice))

    model.add({{choice([Dropout, SpatialDropout2D])}}({{choice([(0.25, 0.375, 0.5)])}}))
    
    
    # Block 2
    model.add(B2_Conv_Layer_Choice)
    model.add(Activation_Choice)
    
    model.add(B2_Conv_Layer_Choice)
    model.add(Activation_Choice)
    
    if {{choice(['two', 'three'])}} == 'three':
        
        model.add(B2_Conv_Layer_Choice)
        model.add(Activation_Choice)
        
    if Pool_Choice == 'MaxPool':
        
        model.add(MaxPooling2D(pool_size = Max_Pool_Size_Choice, 
                               strides = Strides_Choice))

    model.add(Dropout_Choice({{choice([(0.25, 0.375, 0.5)])}}))
    
    
    # Block 3
    model.add(B3_Conv_Layer_Choice)
    model.add(Activation_Choice)
    
    model.add(B3_Conv_Layer_Choice)
    model.add(Activation_Choice)
    
    if {{choice(['two', 'three'])}} == 'three':
        
        model.add(B3_Conv_Layer_Choice)
        model.add(Activation_Choice)
        
    if Max_Pool_Choice == 'MaxPool':
        
        model.add(MaxPooling2D(pool_size = Max_Pool_Size_Choice, 
                               strides = Strides_Choice))

    model.add(Dropout_Choice({{choice([(0.25, 0.375, 0.5)])}}))
    
    
    # Output Block
    
    if Pool_Choice == 'Global':
        
        model.add(GlobalAveragePooling2D())
        
    model.add(Dense(Dense_Neurons_Choice))
    model.add(Activation_Choice)
    
    if {{choice(['one', 'two'])}} == 'two':
        
        model.add(Dense(Dense_Neurons_Choice))
        model.add(Activation_Choice)
        
    model.add(Dense(15))
    model.add(Activation({{choice(['sigmoid', 'softmax'])}}))
    
    model.compile(loss = 'binary_crossentropy', 
                  optimizer = optimizers.RMSprop(lr = Learning_Rate_Choice), 
                  metrics = ['categorical_accuracy'])
    
    model.fit(training_img, training_labels,
              batch_size = Batch_Size_Choice,
              epochs = 10,
              verbose = 0,
              validation_split = 0.1)
    
    Predictions = model.predict(test_img)
    Predictions = Predictions.astype(dtype = 'int32')
    
    Accuracy = true_accuracy(Predictions, test_labels)
    
    return {'loss': -Accuracy, 'status': STATUS_OK, 'model': model}

### VGG_Doppleganger

In [ ]:
# Directly taken from VGG16 with 2D Spatial Dropout, Dropout, and fewer Dense layers
'''
categorical_accuracy
'''
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 20, validation_split = 0.1, batch_size = 128, verbose = 1)

history_str = 'VGG_Doppleganger_v3_history'
model_str   = 'vanilla_reg_VGG_v4_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Removed Conv2D layer in Block 3 and moved SpDrop layer to Block 1
'''
categorical_accuracy
'''
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 20, validation_split = 0.1, batch_size = 128, verbose = 1)

history_str = 'VGG_Doppleganger_v3_history'
model_str   = 'vanilla_reg_VGG_v4_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Removed Conv2D layer in Block 3 and moved SpDrop layer to Block 1
# v3: Using Categorical Accuracy as metrics
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 20, validation_split = 0.1, batch_size = 128)

history_str = 'VGG_Doppleganger_v3_history'
model_str   = 'vanilla_reg_VGG_v4_30e'
    
save_model(model_obj, history_str, model_str)

### vanilla_VGG_Dropouts

In [ ]:
# Based on VGG with half reg'd layers and Dropout layers
'''
categorical_accuracy
'''
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_VGG_Dropouts_history'
model_str   = 'vanilla_VGG_Dropouts_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_VGG_Dropouts_v2_history'
model_str   = 'vanilla_VGG_Dropouts_v2_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
# v3: Added reg's to all layers
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_VGG_Dropouts_v3_history'
model_str   = 'vanilla_VGG_Dropouts_v3_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
# v3: Added reg's to all layers
# v4: Changed Regularizer Weight Penalty (0.001 -> 0.005)
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.5, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_VGG_Dropouts_v4_history'
model_str   = 'vanilla_VGG_Dropouts_v4_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
# v3: Added reg's to all layers
# v4: Changed Regularizer Weight Penalty (0.001 -> 0.005)
# v5: Added Dropout to all layers previously without, changed percentage
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.25, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.SpatialDropout2D(0.25, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.375))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.375))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.375))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_VGG_Dropouts_v5_history'
model_str   = 'vanilla_VGG_Dropouts_v5_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added 2D Sp Dropout to Block 1
# v3: Added reg's to all layers
# v4: Changed Regularizer Weight Penalty (0.001 -> 0.005)
# v5: Added Dropout to all layers previously without, changed percentage
# v6: Added Batch Normalization layers to each Block
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.SpatialDropout2D(0.25, data_format = 'channels_last'))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.SpatialDropout2D(0.25, data_format = 'channels_last'))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.375))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.375))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.005), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.375))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))


model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_VGG_Dropouts_v6_history'
model_str   = 'vanilla_VGG_Dropouts_v6_30e'
    
save_model(model_obj, history_str, model_str)

### vanilla_reg_VGG

In [ ]:
# Modeled after VGGNet with half reg'd layers
vanilla_reg_VGG = models.Sequential()

# Block 1
vanilla_reg_VGG.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
vanilla_reg_VGG.add(layers.Conv2D(16, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Block 2
vanilla_reg_VGG.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(32, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Block 3
vanilla_reg_VGG.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(64, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Block 4
vanilla_reg_VGG.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(128, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Block 5
vanilla_reg_VGG.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
vanilla_reg_VGG.add(layers.Conv2D(256, (3, 3), activation='relu'))
vanilla_reg_VGG.add(layers.MaxPooling2D((2, 2)))

# Output Block
vanilla_reg_VGG.add(layers.Flatten())
vanilla_reg_VGG.add(layers.Dense(64, activation='relu'))
vanilla_reg_VGG.add(layers.Dense(15, activation='sigmoid'))

vanilla_reg_VGG.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = vanilla_reg_VGG.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_reg_VGG_history'
model_str   = 'vanilla_reg_VGG_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added reg's to all layers
'''
categorical_accuracy
'''
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_reg_VGG_v2_history'
model_str   = 'vanilla_reg_VGG_v2_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added reg's to all layers
# v3: Changed No. of Outputs in Conv and Dense layers in Block 5 and Output
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Block 5
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_reg_VGG_v3_history'
model_str   = 'vanilla_reg_VGG_v3_30e'
    
save_model(model_obj, history_str, model_str)

In [ ]:
# v2: Added reg's to all layers
# v3: Changed No. of Outputs in Conv and Dense layers in Block 5 and Output
# v4: Added Batch Normalization to all layers
model = models.Sequential()

# Block 1
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1)))
model.add(layers.Conv2D(16, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Block 2
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Block 3
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Block 4
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Block 5
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Output Block
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(15, activation='sigmoid'))

model.compile(optimizer = optimizers.RMSprop(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['categorical_accuracy'])

model_obj = model.fit(training_img, training_labels, epochs = 30, validation_split = (1 / 9), batch_size = 256, verbose = 1)

history_str = 'vanilla_reg_VGG_v4_history'
model_str   = 'vanilla_reg_VGG_v4_30e'
    
save_model(model_obj, history_str, model_str)